In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_recall_fscore_support




def limpar_texto_simplificado(texto):
    texto = texto.lower()  # Convertendo para minúculas
    # Remover caracteres especiais e números
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()  # Remover espaços extras
    return texto


# Carregando arquivo excel
file_path = 'Base_classificacao_VIDA.xlsx'
data = pd.read_excel(file_path)

# Aplicar a função de limpeza
data['descricao_reclamacao_limpa'] = data['descricao_reclamacao'].apply(
    limpar_texto_simplificado)

# Separar os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    data['descricao_reclamacao_limpa'], data['categoria'], test_size=0.2, random_state=42)

# Vetorização e Simplificação para unigramas
vectorizer = TfidfVectorizer(max_df=0.5, ngram_range=(1, 1))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Modelo RandomForestClassifier
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train_tfidf, y_train)

# Avaliar o modelo
predicoes = modelo.predict(X_test_tfidf)
report = classification_report(y_test, predicoes)

# Gerar métricas de precisão, recall, f1-score para cada categoria
metrics = precision_recall_fscore_support(
    y_test, predicoes, labels=np.unique(predicoes))
metrics_df = pd.DataFrame(metrics, index=[
                          'Precision', 'Recall', 'F1-Score', 'Support'], columns=np.unique(predicoes)).T

# Verificar se todos os F1-Scores são superiores a 75%
f1_scores = metrics_df['F1-Score'] > 0.75
todos_superiores_75 = f1_scores.all()

print(report)
print("Todos os F1-Scores são superiores a 75%:", todos_superiores_75)

                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.79      0.87      0.83      1055
            Hipotecas / Empréstimos       0.85      0.86      0.86       718
                             Outros       0.92      0.47      0.62       437
       Roubo / Relatório de disputa       0.81      0.80      0.81       958
         Serviços de conta bancária       0.79      0.89      0.84      1047

                           accuracy                           0.81      4215
                          macro avg       0.83      0.78      0.79      4215
                       weighted avg       0.82      0.81      0.81      4215

Todos os F1-Scores são superiores a 75%: False
